In [4]:
!pip install --upgrade langchain openai  -q
!pip install sentence_transformers -q
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!pip install -U langchain-community -q
!pip install pillow
!pip install requests
!pip install python-pptx
!pip install gensim==3.8.3
!pip install keybert
!pip install requests Pillow
!pip install google_images_download
!pip install google-search-results
!pip install groq
!pip install PyPDF2
!pip install pinecone -q
!sudo apt-get update
!sudo apt-get install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra

from pptx import Presentation
from pptx.util import Pt, Inches
from PIL import Image
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from io import BytesIO
import requests
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize KeyBERT
from keybert import KeyBERT
kw_model = KeyBERT()

nltk.download('punkt_tab')

from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
  Using cached gensim-3.8.3.tar.gz (23.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (gensim)
Hit:1 http://security.ubuntu.co

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Vector DB Connection

In [38]:
!pip install langchain_pinecone
import getpass
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_pinecone import Pinecone as LangchainPinecone  # Correct import

#074e0d9a-ab5e-48bf-8eae-9effae335521
# Pinecone API key
os.environ["PINECONE_API_KEY"] = "074e0d9a-ab5e-48bf-8eae-9effae335521"
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Define your index name
index_name = "newdata"  # Change if desired

# Check for existing indexes
existing_indexes = [index_info.name for index_info in pc.list_indexes()]

# Create the index if it does not exist
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust this to match your embeddings' dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

    # Wait until the index is ready
    while not pc.describe_index(index_name).status["ready"]:
        print("Waiting for the index to be ready...")
        time.sleep(1)

# Connect to the index as a Pinecone instance
index = pc.Index(index_name)

# Print connection success message
print(f"Successfully connected to the index: {index_name}")

# Define embeddings model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Correct way to create LangChain Pinecone index
langchain_index = LangchainPinecone(
    index=index,  # Pass the correct Pinecone instance
    embedding=embeddings,
    text_key="text"  # Required argument for storing/retrieving text
)

# Output verification
print(f"Successfully created or connected to the LangChain index: {langchain_index}")


Successfully connected to the index: newdata
Successfully created or connected to the LangChain index: <langchain_pinecone.vectorstores.Pinecone object at 0x792e296a34d0>


#Relevant Document Extraction from Vector DB

In [39]:
def get_similar_docs(query, k=20, score=True):
    if score:
        similar_docs = langchain_index.similarity_search_with_score(query, k=k)  # Use langchain_index
    else:
        similar_docs = langchain_index.similarity_search(query, k=k)  # Use langchain_index
    return similar_docs

Extraction of answer from LLM

In [75]:
###############################################  WITH GROQ #############################################################
from groq import Groq
import os

class GroqClient:
    def __init__(self, api_key):
        self.client = Groq(api_key=api_key)

    def get_completion(self, query, content ,max_tokens=3500):
        prompt = self._construct_prompt(query, content)

        try:
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful AI assistant specialized in providing detailed technical explanations."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="deepseek-r1-distill-llama-70b",
                temperature=0.5,
                max_tokens=max_tokens,
                top_p=0.9,
                stream=False
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            raise Exception(f"API request failed: {str(e)}")

    def _construct_prompt(self, query, content):
        return f'''
Question: {query}

Content: {content}


Provide a comprehensive explanation strictly following this structure. Always use "##" to introduce each section:

## 1. **Concept Overview**
  • Core definition and purpose
  • Key principles
  • Relationship to broader computing concepts

## 2. **Technical Components**
  • Primary elements and their roles
  • Relationships and interactions
  • Implementation details
  • Mathematical Equations
  • Core algorithms/procedures (if applicable)

## 3. **Working Mechanism**
  • Step-by-step operational flow
  • Critical processes and transformations
  • Resource management (if applicable)
  • Exception handling (if applicable)

## 4. **Implementation Example**
  • Use case scenario
  • Code implementation or technical design
  • Step-by-step execution
  • Output analysis

## 5. **Best Practices**
  • Design considerations
  • Optimization techniques
  • Common pitfalls
  • Performance implications

## 6. **Applications**
  • Real-world use cases
  • Industry applications
  • Integration patterns
  • Variations and alternatives

## 7. **Evaluation**
  • Performance metrics
  • Testing approaches
  • Debugging strategies
  • Optimization opportunities

## 8. **Practice Problems**
  • Concept verification questions
  • Implementation challenges
  • Problem-solving scenarios
  • Solutions with explanations

### Format Requirements:
- Always use "##" to introduce each section header. Avoid using any other format.
- Use bullet points for clarity
- Show all mathematical steps using proper equation formatting ($...$)
- Include clear variable definitions after each equation
- Write formulas using LaTeX formatting inside $...$
- For matrices use: $\begin{{bmatrix}} a & b \\ c & d \end{{bmatrix}}$
- For fractions use: $\frac{{numerator}}{{denominator}}$
- Use ■ for numbered equations and • for regular points
- Demonstrate practical interpretation
- Connect to real applications

### Equation Guidelines:
- Enclose equations in $...$ format
- Use proper LaTeX notation for mathematical expressions
- Use $\sum$ for summations
- Define each variable after presenting equations
- Number important equations using ■
- Show step-by-step derivations with clear explanations

### Code Guidelines:
- Use LaTeX verbatim environment for code blocks:
  \begin{{verbatim}}
  code here
  \end{{verbatim}}

- For inline code use \texttt{{code}}
- For syntax highlighting:
  \begin{{lstlisting}}[language=Python]
  code here
  \end{{lstlisting}}
- Include comments explaining code functionality
- Show output examples where applicable
'''

if __name__ == "__main__":
    # api_key = os.environ.get("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")
    # client = GroqClient(api_key)
    query = "LSTM"
    content = get_similar_docs(query)

    os.environ["GROQ_API_KEY"] = "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"
    api_key = os.environ.get("GROQ_API_KEY")
    client = GroqClient(api_key)

    try:
        answer = client.get_completion(query, content)
        print("Answer:", answer)
    except Exception as e:
        print("Error:", str(e))

Answer: <think>
Okay, so I need to explain LSTM based on the provided content. Let me start by understanding what the user is asking for. They want a comprehensive explanation of LSTM following a specific structure with eight sections. Each section has its own points, and they want me to use "##" for headers and bullet points. They also provided a lot of content from various documents, which I can use as a reference.

First, I'll look through the documents to gather information about LSTM. I see mentions of Graves, Hochreiter, and Schmidhuber, which I remember are key researchers in LSTM development. The documents talk about the limitations of traditional RNNs, like the vanishing gradient problem, and how LSTMs address this with memory cells and gates. There's also discussion on the components of LSTM, such as input, output, and forget gates, and how they interact.

For the concept overview, I should define what LSTM is, its purpose, key principles, and how it relates to other computin

In [76]:
def extract_answer(text):
   """Extract the answer, removing everything before 'Show output examples where applicable'"""
   # Find the index of the end section
   start_index = text.find("</think>")

   if start_index == -1:
       return text.strip()

   return text[start_index + len("</think>"):].strip()

# Usage
clean_answer = extract_answer(answer)
print(clean_answer)

## 1. **Concept Overview**
- **Core Definition and Purpose**:  
  Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) designed to handle the vanishing gradient problem in traditional RNNs. They are primarily used for sequence modeling tasks, such as language modeling, speech recognition, and time-series prediction.  
- **Key Principles**:  
  - LSTMs introduce memory cells and gates (input, output, and forget gates) to control the flow of information.  
  - These components allow the network to learn long-term dependencies by selectively retaining or discarding information.  
- **Relationship to Broader Computing Concepts**:  
  - LSTMs are a part of deep learning and neural networks, extending the capabilities of RNNs.  
  - They are particularly useful for sequential data, where temporal relationships are important.

## 2. **Technical Components**
- **Primary Elements and Their Roles**:  
  - **Memory Cell (Cell State)**: Acts as the internal memory of

#Format the content

In [77]:
import re

def format_text(text):
    """
    Format text with improved numbering system and removal of empty items
    """
    lines = text.split('\n')
    formatted_lines = []
    level_counters = {}
    current_section_level = 0

    def get_indentation(line):
        return len(line) - len(line.lstrip())

    def clean_content(content):
        """Remove dashes and clean up content"""
        # Remove standalone dashes
        content = re.sub(r'^--+$', '', content)
        # Remove dashes with spaces around them
        content = re.sub(r'\s*--+\s*', ' ', content)
        # Clean up extra spaces
        content = re.sub(r'\s+', ' ', content)
        return content.strip()

    def process_numbered_item(line):
        """Process numbered items and return None if empty after cleaning"""
        # Match numbered items with or without bold markers
        match = re.match(r'^(\d+\.\s*)((?:\*\*)?[^:]*(?:\*\*)?:?\s*)?(.*)$', line.strip())
        if match:
            number, title, content = match.groups()
            title = title or ''
            content = content or ''

            # Clean the content
            cleaned_content = clean_content(content)
            cleaned_title = title.strip()

            # If both title and content are empty (or just dashes), return None
            if not cleaned_title and not cleaned_content:
                return None

            # Reconstruct the line
            return f"{number}{title}{cleaned_content}".rstrip()
        return line

    for i, line in enumerate(lines):
        if not line.strip():
            formatted_lines.append(line)
            continue

        indent = get_indentation(line)
        stripped_line = line.strip()

        # Convert single asterisks to double
        line = re.sub(
            r'(?<![\*])\*(?![\*])([^\*]+)(?<![\*])\*(?![\*])',
            r'**\1**',
            line
        )

        # Handle section headers
        if stripped_line.startswith('##'):
            level_counters.clear()
            current_section_level = indent
            heading_parts = stripped_line.split(' ', 1)
            if len(heading_parts) > 1:
                line = f"{' ' * indent}{heading_parts[0]} {clean_content(heading_parts[1])}"
            formatted_lines.append(line)

        # Handle numbered items
        elif re.match(r'^\d+\.', stripped_line):
            spaces = ' ' * indent
            processed_line = process_numbered_item(stripped_line)
            if processed_line is not None:  # Only add if there's actual content
                line = f"{spaces}{processed_line}"
                formatted_lines.append(line)

        # Handle other bullet points (###, ####, •, -, ■)
        elif any(stripped_line.startswith(marker) for marker in ('###', '####', '•', '-', '■')):
            if indent not in level_counters:
                level_counters[indent] = 1
            else:
                level_counters[indent] += 1

            level_counters = {k: v for k, v in level_counters.items() if k <= indent}
            number = level_counters[indent]
            spaces = ' ' * indent

            # Extract content based on marker type
            if stripped_line.startswith('####'):
                content = stripped_line[4:]
            elif stripped_line.startswith('###'):
                content = stripped_line[3:]
            else:
                content = stripped_line[1:]

            content = clean_content(content.strip())
            if content:  # Only add if there's content
                line = f"{spaces}{number}. {content}"
                formatted_lines.append(line)

        else:
            # Regular text lines
            current_section_level = indent
            cleaned_line = clean_content(stripped_line)
            if cleaned_line:  # Only add if there's content
                line = f"{' ' * indent}{cleaned_line}"
                formatted_lines.append(line)

    # Clean up consecutive empty lines
    cleaned_lines = []
    prev_empty = False
    for line in formatted_lines:
        if not line.strip():
            if not prev_empty:
                cleaned_lines.append(line)
            prev_empty = True
        else:
            cleaned_lines.append(line)
            prev_empty = False

    return '\n'.join(cleaned_lines)

def process_file(input_text):
    """Process the entire file and apply formatting"""
    return format_text(input_text)

if __name__ == "__main__":


    result = process_file(clean_answer)
    print(result)

## 1. **Concept Overview**
1. **Core Definition and Purpose**:
  Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) designed to handle the vanishing gradient problem in traditional RNNs. They are primarily used for sequence modeling tasks, such as language modeling, speech recognition, and time-series prediction.
2. **Key Principles**:
  1. LSTMs introduce memory cells and gates (input, output, and forget gates) to control the flow of information.
  2. These components allow the network to learn long-term dependencies by selectively retaining or discarding information.
3. **Relationship to Broader Computing Concepts**:
  1. LSTMs are a part of deep learning and neural networks, extending the capabilities of RNNs.
  2. They are particularly useful for sequential data, where temporal relationships are important.

## 2. **Technical Components**
1. **Primary Elements and Their Roles**:
  1. **Memory Cell (Cell State)**: Acts as the internal memory of the LS

#Save Content in txt File

In [78]:
file_path = "content.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(result)

#Formatted Content to Json

In [79]:
import re
import json

class MarkdownToJsonConverter:
    def __init__(self):
        self.result = {}
        self.current_section = None
        self.current_item = None
        self.current_subitem = None
        self.in_code_block = False
        self.code_content = []
        self.code_language = None
        self.indentation_level = 0
        self.last_item_at_level = {}  # Track last item at each indentation level

    def get_indentation_level(self, line):
        """Calculate the indentation level of a line"""
        return len(line) - len(line.lstrip())

    def parse_section(self, line):
        """Parse a section header line"""
        if self.in_code_block:
            return False

        # Match section headers with more flexible format
        section_match = re.match(r'^##\s*(\d+)\.\s*(?:\*\*)?([^*]+?)(?:\*\*)?$', line.strip())

        if section_match:
            section_num = section_match.group(1)
            section_title = section_match.group(2).strip()
            self.current_section = {
                "type": "section",
                "number": section_num,
                "title": section_title,
                "items": []
            }
            self.result[f"Section {section_num}"] = self.current_section
            self.current_item = None
            self.current_subitem = None
            self.last_item_at_level = {}  # Reset item tracking for new section
            return True
        return False

    def find_parent_item(self, indentation):
        """Find the appropriate parent item based on indentation"""
        for level in sorted(self.last_item_at_level.keys(), reverse=True):
            if level < indentation:
                return self.last_item_at_level[level]
        return None

    def parse_item(self, line):
        """Parse a numbered item line with improved title and nested content handling"""
        if self.in_code_block:
            return False

        stripped_line = line.lstrip()
        indentation = self.get_indentation_level(line)

        # Handle both numbered items and titled content
        item_match = re.match(r'^(\d+)\.\s*(?:\*\*([^*]+?)\*\*:?)?\s*(.+)?$', stripped_line)
        title_match = re.match(r'^\*\*([^*]+?)\*\*:?\s*(.+)?$', stripped_line)

        if item_match:
            item_num = item_match.group(1)
            item_title = item_match.group(2)
            item_content = item_match.group(3)

            new_item = {
                "type": "item",
                "number": item_num,
                "indentation": indentation,
                "content": []
            }

            if item_title:
                new_item["title"] = item_title.strip()

            if item_content:
                new_item["content"].append({
                    "type": "text",
                    "content": item_content.strip()
                })

            parent_item = self.find_parent_item(indentation)

            if parent_item:
                if "subitems" not in parent_item:
                    parent_item["subitems"] = []
                parent_item["subitems"].append(new_item)
                self.current_subitem = new_item
            else:
                if self.current_section:
                    self.current_section["items"].append(new_item)
                self.current_item = new_item
                self.current_subitem = None

            self.last_item_at_level[indentation] = new_item
            return True

        elif title_match:
            title = title_match.group(1).strip()
            content = title_match.group(2)

            new_item = {
                "type": "item",
                "title": title,
                "indentation": indentation,
                "content": []
            }

            if content:
                new_item["content"].append({
                    "type": "text",
                    "content": content.strip()
                })

            parent_item = self.find_parent_item(indentation)

            if parent_item:
                if "subitems" not in parent_item:
                    parent_item["subitems"] = []
                parent_item["subitems"].append(new_item)
                self.current_subitem = new_item
            else:
                if self.current_section:
                    self.current_section["items"].append(new_item)
                self.current_item = new_item

            self.last_item_at_level[indentation] = new_item
            return True

        return False

    def handle_code_block(self, line):
        """Handle code block markers and content with improved nesting support"""
        stripped_line = line.lstrip()
        indentation = self.get_indentation_level(line)

        if stripped_line.startswith('```'):
            if not self.in_code_block:
                # Start of code block
                self.in_code_block = True
                self.code_language = stripped_line[3:].strip()
                self.code_content = []
                self.code_indentation = indentation
            else:
                # End of code block
                target_item = self.find_parent_item(self.code_indentation)
                if not target_item:
                    target_item = self.current_item

                if target_item and self.code_content:
                    content = '\n'.join(
                        line[self.code_indentation:] if line.startswith(' ' * self.code_indentation)
                        else line
                        for line in self.code_content
                    )
                    target_item["content"].append({
                        "type": "code",
                        "language": self.code_language or "",
                        "content": content,
                        "indentation": self.code_indentation
                    })
                self.in_code_block = False
                self.code_content = []
                self.code_language = None
            return True

        if self.in_code_block:
            self.code_content.append(line)  # Store the line with original indentation
            return True

        return False

    def handle_text_content(self, line):
        """Handle regular text content with improved indentation awareness"""
        if not self.in_code_block and line.strip():
            indentation = self.get_indentation_level(line)
            content = line.strip()

            target_item = self.find_parent_item(indentation)
            if not target_item:
                target_item = self.current_item

            if target_item and content:
                target_item["content"].append({
                    "type": "text",
                    "content": content,
                    "indentation": indentation
                })
            return True
        return False

    def convert(self, markdown_text):
        """Convert markdown text to JSON structure with improved handling of nested content"""
        self.result = {}
        self.last_item_at_level = {}
        lines = markdown_text.split('\n')

        for line in lines:
            # Skip empty lines outside of code blocks
            if not self.in_code_block and not line.strip():
                continue

            # Handle code blocks first
            if self.handle_code_block(line):
                continue

            # Try to parse as section
            if self.parse_section(line):
                continue

            # Try to parse as item
            if self.parse_item(line):
                continue

            # Handle remaining text content
            self.handle_text_content(line)

        # Clean up empty sections and items
        for section_key, section in list(self.result.items()):
            if not section["items"]:
                del self.result[section_key]
                continue

            # Clean up items recursively
            self._clean_items(section["items"])

        return self.result

    def _clean_items(self, items):
        """Recursively clean up empty content and subitems"""
        for item in items:
            if "content" in item and not item["content"]:
                del item["content"]

            if "subitems" in item:
                self._clean_items(item["subitems"])
                if not item["subitems"]:
                    del item["subitems"]

            # Remove indentation from final output
            if "indentation" in item:
                del item["indentation"]



def convert_markdown_file(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            content = f.read()

        converter = MarkdownToJsonConverter()
        json_output = converter.convert(content)

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(json_output, f, indent=2, ensure_ascii=False)

        return json_output

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

if __name__ == "__main__":
    result = convert_markdown_file("content.txt", "output.json")

    # Print first section as sample
    if result:
        first_section = next(iter(result.values()))
        print("Sample output (first section):")
        print(json.dumps(first_section, indent=2))

Sample output (first section):
{
  "type": "section",
  "number": "1",
  "title": "Concept Overview",
  "items": [
    {
      "type": "item",
      "number": "1",
      "content": [
        {
          "type": "text",
          "content": "Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) designed to handle the vanishing gradient problem in traditional RNNs. They are primarily used for sequence modeling tasks, such as language modeling, speech recognition, and time-series prediction.",
          "indentation": 2
        }
      ],
      "title": "Core Definition and Purpose"
    },
    {
      "type": "item",
      "number": "2",
      "title": "Key Principles",
      "subitems": [
        {
          "type": "item",
          "number": "1",
          "content": [
            {
              "type": "text",
              "content": "LSTMs introduce memory cells and gates (input, output, and forget gates) to control the flow of information."
          

##Images Getter


In [80]:
from groq import Groq

class GroqClient:
    def __init__(self, api_key):
        self.client = Groq(api_key=api_key)

    def get_completion(self, query, max_tokens=2500):
        prompt = self._construct_prompt(query)

        try:
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a technical visualization expert specialized in extracting image search keywords."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="deepseek-r1-distill-llama-70b",
                temperature=0.5,
                max_tokens=max_tokens,
                top_p=0.7,
                stream=False
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            raise Exception(f"API request failed: {str(e)}")

    def _construct_prompt(self, content):
        return f"""Extract specific image search keywords from the following content:

{content}

Your task: Generate 5-7 image search keywords that would help find educational diagrams and technical visualizations related to the main concepts in this content.

Focus on identifying keywords for:
1. System/concept architecture diagrams
2. Process flows and sequences
3. Component interactions
4. Implementation details
5. Working mechanisms
6. Step-by-step procedures
7. Technical examples

Keyword Generation Rules:
1. Each keyword MUST use hyphens between words
2. Each keyword MUST end with exactly ONE of these suffixes:
   - -visualization
   - -diagram
   - -illustration
   - -example
   - -steps

Key Requirements:
- Use specific technical terms from the content
- Include major concepts and processes
- Avoid generic/non-specific terms
- Keywords must directly relate to main topics
- Each keyword should help find relevant technical diagrams

Output Format:
ONLY provide a comma-separated list of keywords.
Example: concept-architecture-diagram, process-flow-visualization, component-interaction-illustration

KEYWORDS (comma-separated):"""

def extract_keywords(response_text):
    """
    Extracts and validates keywords from the API response.
    """
    try:
        valid_suffixes = [
            '-visualization',
            '-diagram',
            '-illustration',
            '-example',
            '-steps'
        ]

        # Get the comma-separated keywords
        keywords = response_text.strip()

        # Clean up and validate the keywords
        cleaned_keywords = []
        for keyword in keywords.split(','):
            keyword = keyword.strip()

            # Validation checks
            if (keyword and                # Not empty
                '-' in keyword and         # Contains hyphens
                any(keyword.endswith(suffix) for suffix in valid_suffixes) and  # Has valid suffix
                keyword.count('-') >= 2):  # Has at least 2 hyphens

                cleaned_keywords.append(keyword)

        # Return 5-7 keywords
        cleaned_keywords = cleaned_keywords[:7] if len(cleaned_keywords) > 7 else cleaned_keywords

        if len(cleaned_keywords) < 5:
            print(f"Not enough valid keywords found: {len(cleaned_keywords)}")
            return ""

        return ', '.join(cleaned_keywords)

    except Exception as e:
        print(f"Error extracting keywords: {str(e)}")
        return ""


    # Initialize client with your API key
client = GroqClient("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")


try:
        # Get completion from Groq
        response = client.get_completion(result)

        # Extract and validate keywords
        keywords = extract_keywords(response)

        print("Generated Image Search Keywords:")
        print(keywords)

except Exception as e:
        print(f"Error: {str(e)}")




Generated Image Search Keywords:
and applications.
</think>

lstm-architecture-diagram, lstm-memory-cell-diagram, gate-interactions-visualization, lstm-working-mechanism-illustration, cell-state-update-steps, forward-pass-visualization, bidirectional-lstm-diagram


In [81]:
keywords = extract_answer(keywords)

In [82]:
print(keywords)

lstm-architecture-diagram, lstm-memory-cell-diagram, gate-interactions-visualization, lstm-working-mechanism-illustration, cell-state-update-steps, forward-pass-visualization, bidirectional-lstm-diagram


In [83]:
def extract_keywords(response_text):
    """
    Extracts keywords from the API response by separating them using commas.
    If the response includes a header, it is removed before splitting.
    """
    try:
        # Remove header if present
        if "KEYWORDS (comma-separated):" in response_text:
            response_text = response_text.split("KEYWORDS (comma-separated):")[-1].strip()

        # Split on commas and remove any extra whitespace
        keywords_list = [keyword.strip() for keyword in response_text.split(",") if keyword.strip()]
        return keywords_list
    except Exception as e:
        print(f"Error extracting keywords: {str(e)}")
        return []

# Example usage:

keywords = extract_keywords(keywords)
print("Extracted Keywords List:", keywords)


Extracted Keywords List: ['lstm-architecture-diagram', 'lstm-memory-cell-diagram', 'gate-interactions-visualization', 'lstm-working-mechanism-illustration', 'cell-state-update-steps', 'forward-pass-visualization', 'bidirectional-lstm-diagram']


In [84]:
import os
import re
import requests
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor

def scrape_images(keyword, num_images=1):
    """Scrape images from GeeksForGeeks using SERP API"""
    api_key = "df7729cfcc6f85cfea8e18cb9f13ce5180a3ea6e1c22e2e5f3de3ea16bd6e40b"

    params = {
        "engine": "google_images",
        "q": keyword,
        "google_domain": "google.com",
        "gl": "us",
        "hl": "en",
        "api_key": api_key,
    }

    response = requests.get("https://serpapi.com/search.json", params=params)

    if response.status_code == 200:
        data = response.json()
        images = []

        if "images_results" in data:
            for image in data["images_results"]:
                if 'geeksforgeeks' in image.get('source', '').lower():
                    images.append(image["original"])
                    if len(images) >= num_images:
                        break
            return images

        return []
    else:
        print(f"Failed to retrieve images. Status code: {response.status_code}")
        return []
images = set()

try:
    for kw in keywords:
        print(f"\nImages for '{kw}':")
        image_urls = scrape_images(kw)
        print(image_urls)
        if image_urls:  # Ensure there is at least one URL
            images.add(image_urls[0])
except Exception as e:
    print(f"Error: {str(e)}")

print("Collected image URLs:", images)





Images for 'lstm-architecture-diagram':
['https://media.geeksforgeeks.org/wp-content/uploads/20240208053129/lstm.webp']

Images for 'lstm-memory-cell-diagram':
['https://media.geeksforgeeks.org/wp-content/uploads/20240208053129/lstm.webp']

Images for 'gate-interactions-visualization':
[]

Images for 'lstm-working-mechanism-illustration':
['https://media.geeksforgeeks.org/wp-content/uploads/20240208053129/lstm.webp']

Images for 'cell-state-update-steps':
['https://media.geeksforgeeks.org/wp-content/uploads/20240208104902/bruh.webp']

Images for 'forward-pass-visualization':
['https://media.geeksforgeeks.org/wp-content/uploads/20240425175947/Capture.PNG']

Images for 'bidirectional-lstm-diagram':
['https://media.geeksforgeeks.org/wp-content/uploads/20230529103946/Bidirectional-LSTM-(1)-660.jpg']
Collected image URLs: {'https://media.geeksforgeeks.org/wp-content/uploads/20230529103946/Bidirectional-LSTM-(1)-660.jpg', 'https://media.geeksforgeeks.org/wp-content/uploads/20240425175947/Ca

In [85]:
import json

def merge_latex_fragments(content_items):
    """
    Merge adjacent text items that form a single LaTeX expression.
    Supports delimiters: '$', '$$', '\\[' and '\\]'.
    """
    merged_items = []
    i = 0
    # Mapping of opening to closing delimiters.
    delimiters = {"\\[": "\\]", "$": "$", "$$": "$$"}

    while i < len(content_items):
        item = content_items[i]
        # Check if this text item is an opening delimiter exactly.
        if item.get("type") == "text" and item.get("content") in delimiters:
            open_delim = item["content"]
            close_delim = delimiters[open_delim]
            indentation = item.get("indentation", 0)
            math_block = open_delim  # start with the opening delimiter
            i += 1
            # Collect all subsequent text items until the matching closing delimiter is found.
            while i < len(content_items):
                next_item = content_items[i]
                # If we hit the closing delimiter exactly, add it and break.
                if next_item.get("type") == "text" and next_item.get("content") == close_delim:
                    math_block += next_item["content"]
                    i += 1
                    break
                else:
                    # Otherwise, concatenate the text.
                    math_block += next_item.get("content", "")
                    i += 1
            # Append the merged LaTeX block as a single item.
            merged_items.append({
                "type": "text",
                "content": math_block,
                "indentation": indentation
            })
        else:
            # Otherwise, just keep the item as is.
            merged_items.append(item)
            i += 1
    return merged_items

def process_json_obj(obj):
    """
    Recursively traverse the JSON object.
    When a 'content' list is found, merge its LaTeX fragments.
    Also processes any 'items' or 'subitems' lists.
    """
    if isinstance(obj, dict):
        # Process a 'content' list if present.
        if "content" in obj and isinstance(obj["content"], list):
            obj["content"] = merge_latex_fragments(obj["content"])
        # Process nested lists under keys like 'items' and 'subitems'.
        for key in ["items", "subitems"]:
            if key in obj and isinstance(obj[key], list):
                for item in obj[key]:
                    process_json_obj(item)
        # Also process any other dictionary values.
        for value in obj.values():
            process_json_obj(value)
    elif isinstance(obj, list):
        for item in obj:
            process_json_obj(item)
    return obj

# Example: load your complete JSON file, process it, and save the result.
if __name__ == "__main__":
    input_file = "output.json"  # your JSON filename
    output_file = "output_merged.json"  # file to save the merged version

    with open(input_file, 'r', encoding="utf-8") as f:
        data = json.load(f)

    # Process the JSON to merge LaTeX fragments.
    processed_data = process_json_obj(data)

    # Optionally, write the processed JSON back to disk.
    with open(output_file, 'w', encoding="utf-8") as f:
        json.dump(processed_data, f, indent=2, ensure_ascii=False)

    print(f"Merged JSON saved as {output_file}")


Merged JSON saved as output_merged.json


#Equations to Latex

In [88]:
import json
import re
import matplotlib.pyplot as plt
from copy import deepcopy

def clean_equation(equation):
    """Extract equation from various formats"""
    # Try different equation delimiters
    patterns = [
        r'\$(.+?)\$',           # $...$
        r'\\\((.+?)\\\)',       # \(...\)
        r'\\text\{(.+?)\}',     # \text{...}
    ]

    for pattern in patterns:
        match = re.search(pattern, equation)
        if match:
            return match.group(1).strip()

    # If no patterns match, return cleaned original text
    return equation.split(":")[1].strip() if ":" in equation else equation.strip()

def render_equation(latex_equation, index):
    """Render equation using matplotlib's math text"""
    file_path = f"equation_{index + 1}.png"

    # Create figure
    plt.figure(figsize=(10, 2), facecolor='white')
    ax = plt.gca()
    ax.set_facecolor('white')

    # Render the equation
    plt.text(0.5, 0.5, f"${latex_equation}$",
             size=14,
             ha='center',
             va='center',
             transform=ax.transAxes)

    # Remove axes
    plt.axis('off')

    # Save with tight layout
    plt.savefig(file_path,
                bbox_inches='tight',
                pad_inches=0.1,
                dpi=300,
                facecolor='white')
    plt.close()

    print(f"Rendered equation saved as '{file_path}'")
    return file_path

def process_equations(data):
    """Process all equations in the JSON data"""
    updated_data = deepcopy(data)

    # Find equations in all sections
    for section in updated_data.values():
        if isinstance(section, dict) and "items" in section:
            for item in section["items"]:
                if "content" in item:
                    # Process direct content
                    for content_item in item["content"]:
                        if isinstance(content_item, dict) and "content" in content_item:
                            text = content_item["content"]
                            if "$" in text or "\\(" in text:
                                print(f"\nProcessing equation:\nRaw: {text}")
                                math_expr = clean_equation(text)
                                print(f"Cleaned: {math_expr}")
                                png_path = render_equation(math_expr,
                                                        hash(text) % 1000)  # Unique index
                                content_item["content"] = png_path
                                content_item["type"] = "equation_image"

                # Process subitems
                if "subitems" in item:
                    for subitem in item["subitems"]:
                        if "content" in subitem:
                            for content_item in subitem["content"]:
                                if isinstance(content_item, dict) and "content" in content_item:
                                    text = content_item["content"]
                                    if "$" in text or "\\(" in text:
                                        print(f"\nProcessing equation:\nRaw: {text}")
                                        math_expr = clean_equation(text)
                                        print(f"Cleaned: {math_expr}")
                                        png_path = render_equation(math_expr,
                                                                hash(text) % 1000)
                                        content_item["content"] = png_path
                                        content_item["type"] = "equation_image"

    return updated_data

if __name__ == "__main__":
    # Load the JSON file
    with open('output.json', 'r') as file:
        data = json.load(file)

    # Process the equations
    updated_data = process_equations(data)

    # Save the updated JSON structure
    with open('output_updated.json', 'w') as file:
        json.dump(updated_data, file, indent=4)
        print("\nUpdated JSON saved to 'output_updated.json'.")


Processing equation:
Raw: Let \( f_t \), \( i_t \), and \( o_t \) represent the forget, input, and output gates, respectively. Let \( c_t \) and \( h_t \) denote the cell state and hidden state at time \( t \). The key equations are:
Cleaned: f_t
Rendered equation saved as 'equation_288.png'

Processing equation:
Raw: Where \( \sigma \) is the sigmoid function, \( \tanh \) is the hyperbolic tangent function, and \( \odot \) denotes element-wise multiplication.
Cleaned: \sigma
Rendered equation saved as 'equation_855.png'

Processing equation:
Raw: \( f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f) \)
Cleaned: f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f)
Rendered equation saved as 'equation_305.png'

Processing equation:
Raw: \( i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i) \)
Cleaned: i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i)
Rendered equation saved as 'equation_523.png'

Processing equation:
Raw: \( o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o) \)
Cleaned: o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o)
Re

In [ ]:
updated_data["Section 2"]["items"]#[["title"]=="Mathematical Equations"]

[{'type': 'item',
  'number': '1',
  'title': 'Primary elements and their roles',
  'subitems': [{'type': 'item',
    'number': '1',
    'content': [{'type': 'text',
      'content': 'Maintains records of which transactions hold locks on which items. It includes fields like data item name, number of read locks, and the transaction holding a write lock.'}],
    'title': 'Lock Table'},
   {'type': 'item',
    'number': '2',
    'content': [{'type': 'text',
      'content': 'Responsible for granting, releasing, and managing locks. It enforces the rules of the locking protocol.'}],
    'title': 'Lock Manager'},
   {'type': 'item',
    'number': '3',
    'content': [{'type': 'text',
      'content': 'Entities that request locks to access data items.'}],
    'title': 'Transactions'}]},
 {'type': 'item',
  'number': '2',
  'content': [{'type': 'text',
    'content': 'Transactions interact with the lock manager to acquire and release locks. The lock table reflects the current state of locks on

#JSON to PPT(DONT RUN THIS. THIS IS PREVIOUS CODE. BELOW THIS CODE IS NEW CODE.)

In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import json

class JSONToPPTXConverter:
    def __init__(self):
        self.prs = Presentation()
        self.toc_items = []
        self.min_points_per_slide = 3
        self.max_points_per_slide = 5

    def add_title_slide(self, title):
        """Add a title slide at the beginning of the presentation."""
        title_slide_layout = self.prs.slide_layouts[0]
        slide = self.prs.slides.add_slide(title_slide_layout)
        title_shape = slide.shapes.title
        title_shape.text_frame.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)
        title_shape.text_frame.paragraphs[0].font.bold = True

    def add_table_of_contents_slide(self):
        """Add a table of contents slide."""
        toc_slide_layout = self.prs.slide_layouts[1]
        slide = self.prs.slides.add_slide(toc_slide_layout)

        title_shape = slide.shapes.title
        title_shape.text_frame.text = "Table of Contents"
        title_shape.text_frame.paragraphs[0].font.size = Pt(40)
        title_shape.text_frame.paragraphs[0].font.bold = True

        body_shape = slide.placeholders[1]
        text_frame = body_shape.text_frame
        text_frame.clear()

        for item in self.toc_items:
            p = text_frame.add_paragraph()
            p.text = f"{item['number']}. {item['title']}"
            p.font.size = Pt(26)
            p.space_after = Pt(6)

    def group_content_by_title(self, title, content):
        """Group content into chunks of 2-3 items per slide, maintaining the same title."""
        if not isinstance(content, list):
            content = [content]

        grouped_slides = []
        current_group = []

        for item in content:
            current_group.append(item)
            if len(current_group) >= self.max_points_per_slide:
                grouped_slides.append((title, current_group))
                current_group = []

        # Handle remaining items
        if current_group:
            if len(current_group) < self.min_points_per_slide and grouped_slides:
                # Try to combine with the last group if it won't exceed max
                last_group = grouped_slides[-1][1]
                if len(last_group) + len(current_group) <= self.max_points_per_slide:
                    last_group.extend(current_group)
                else:
                    grouped_slides.append((title, current_group))
            else:
                grouped_slides.append((title, current_group))

        return grouped_slides

    def add_text_to_frame(self, text_frame, content, font_size=30, font_name='Calibri'):
        """Helper method to add text to a text frame."""
        p = text_frame.add_paragraph()
        p.text = str(content)
        p.font.size = Pt(font_size)
        p.font.name = font_name
        return p

    def create_content_slide(self, title, content_group, slide_layout_idx=1):
        """Create a single content slide with the given title and content group."""
        slide_layout = self.prs.slide_layouts[slide_layout_idx]
        slide = self.prs.slides.add_slide(slide_layout)

        # Add title
        title_shape = slide.shapes.title
        title_shape.text_frame.text = title
        title_para = title_shape.text_frame.paragraphs[0]
        title_para.font.size = Pt(36)
        title_para.font.bold = True

        # Add content
        body_shape = slide.placeholders[1]
        text_frame = body_shape.text_frame
        text_frame.clear()

        for item in content_group:
            if isinstance(item, dict):
                if 'type' in item and item['type'] == 'code':
                    self.add_text_to_frame(
                        text_frame,
                        f"Code ({item.get('language', '')}):\n{item['content']}",
                        font_size=10,
                        font_name='Courier New'
                    )
                elif 'type' in item and item['type'] == 'text':
                    self.add_text_to_frame(text_frame, item['content'])
            else:
                self.add_text_to_frame(text_frame, item)

    def process_section(self, section_data):
        """Process a section and its items, grouping content by title."""
        self.toc_items.append({
            'number': section_data['number'],
            'title': section_data['title']
        })

        section_title = f"Section {section_data['number']}: {section_data['title']}"

        for item in section_data['items']:
            item_title = f"{section_data['title']} - {item.get('title', '')}"

            if 'content' in item:
                # Group content for this item
                grouped_slides = self.group_content_by_title(item_title, item['content'])
                for title, content_group in grouped_slides:
                    self.create_content_slide(title, content_group)

            if 'subitems' in item and item['subitems']:
                for subitem in item['subitems']:
                    subitem_content = subitem.get('content', [])
                    if subitem.get('title'):
                        subitem_title = f"{item_title} - {subitem['title']}"
                    else:
                        subitem_title = f"{item_title} - {subitem.get('number', '')}"

                    if subitem_content:
                        # Group content for this subitem
                        grouped_slides = self.group_content_by_title(subitem_title, subitem_content)
                        for title, content_group in grouped_slides:
                            self.create_content_slide(title, content_group)

    def _move_last_slide_to_index(self, index):
        """Move the last slide to the specified index."""
        sldIdLst = self.prs.slides._sldIdLst
        slides_ids = list(sldIdLst)
        last_slide_id = slides_ids[-1]
        sldIdLst.remove(last_slide_id)
        sldIdLst.insert(index, last_slide_id)

    def convert_json_to_pptx(self, json_data, output_filename='output.pptx'):
        """Convert JSON data to PowerPoint presentation."""
        self.add_title_slide(output_filename.split('.')[0].upper())

        for section_key, section_data in json_data.items():
            if section_data['type'] == 'section':
                self.process_section(section_data)

        self.add_table_of_contents_slide()
        self._move_last_slide_to_index(1)

        self.prs.save(output_filename)

def main():
    with open('output.json', 'r') as file:
        json_data = json.load(file)

    converter = JSONToPPTXConverter()
    converter.convert_json_to_pptx(json_data, query.upper()+'.pptx')

if __name__ == "__main__":
    main()

#JSON to PPT Final

In [37]:
import json
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_AUTO_SIZE

class PPTGenerator:
    def __init__(self):
        self.prs = Presentation("/content/Crop.pptx")
        self.slide_width = self.prs.slide_width
        self.slide_height = self.prs.slide_height
        self.slide_count = 0

        # Color scheme
        self.colors = {
            'title': RGBColor(44, 62, 80),      # Dark blue
            'subtitle': RGBColor(52, 73, 94),    # Lighter blue
            'body': RGBColor(44, 62, 80),        # Dark blue
            'code': RGBColor(46, 204, 113),      # Green
            'slide_number': RGBColor(149, 165, 166)  # Gray
        }

    def add_slide_number(self, slide):
        """Add slide number to the bottom right corner"""
        self.slide_count += 1
        txBox = slide.shapes.add_textbox(
            Inches(9), Inches(6.5), Inches(1), Inches(0.5))
        tf = txBox.text_frame
        p = tf.paragraphs[0]
        p.text = f"Slide {self.slide_count}"
        p.alignment = PP_ALIGN.RIGHT
        p.font.size = Pt(12)
        p.font.color.rgb = self.colors['slide_number']

    def add_title_slide(self, title):
        """Add title slide"""
        slide_layout = self.prs.slide_layouts[0]
        slide = self.prs.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)

        self.add_slide_number(slide)
        return slide

    def format_bullet_point(self, paragraph, level=0):
        """Format bullet point with proper indentation and style"""
        paragraph.level = level
        paragraph.font.size = Pt(25 - level)  # Decrease font size for deeper levels
        paragraph.font.color.rgb = self.colors['body']
        return paragraph

    def add_content_slide(self, title, content_items, is_subsection=False):
        """Add content slide with proper bullet points and overflow management"""
        slide_layout = self.prs.slide_layouts[1]
        current_slide = self.prs.slides.add_slide(slide_layout)

        # Add title
        title_shape = current_slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        # Add content
        content_shape = current_slide.placeholders[1]
        tf = content_shape.text_frame
        tf.word_wrap = True

        # Clear default paragraph
        tf.clear()

        current_height = 0
        max_height = Inches(5)  # Maximum content height

        for item in content_items:
            # Estimate text height
            text_height = len(item.split()) * Pt(25) * 0.3  # Rough estimation

            if current_height + text_height > max_height:
                # Create new slide if content overflows
                current_slide = self.prs.slides.add_slide(slide_layout)
                title_shape = current_slide.shapes.title
                title_shape.text = f"{title} (continued)"
                title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                title_shape.text_frame.paragraphs[0].font.size = Pt(32)

                content_shape = current_slide.placeholders[1]
                tf = content_shape.text_frame
                tf.word_wrap = True
                current_height = 0

                self.add_slide_number(current_slide)

            p = tf.add_paragraph()
            p.text = item
            self.format_bullet_point(p, 1 if is_subsection else 0)
            current_height += text_height

        self.add_slide_number(current_slide)
        return current_slide

    def add_code_slide(self, title, code):
        """Add code slide with proper formatting"""
        slide_layout = self.prs.slide_layouts[1]
        slide = self.prs.slides.add_slide(slide_layout)

        # Add title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        # Add code with text box for better control
        left = Inches(2)
        top = Inches(2.6)
        width = Inches(8)
        height = Inches(5)

        textbox = slide.shapes.add_textbox(left, top, width, height)
        tf = textbox.text_frame
        tf.word_wrap = True

        # Split code into smaller chunks if too long
        code_lines = code.split('\n')
        lines_per_slide = 20

        for i in range(0, len(code_lines), lines_per_slide):
            if i > 0:
                slide = self.prs.slides.add_slide(slide_layout)
                title_shape = slide.shapes.title
                title_shape.text = f"{title} (continued)"
                title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                textbox = slide.shapes.add_textbox(left, top, width, height)
                tf = textbox.text_frame
                tf.word_wrap = True

            chunk = '\n'.join(code_lines[i:i + lines_per_slide])
            p = tf.add_paragraph()
            p.text = chunk
            p.font.name = 'Courier New'
            p.font.size = Pt(11)
            p.font.color.rgb = self.colors['code']

            self.add_slide_number(slide)

        return slide

    def process_json_to_ppt(self, json_data):
        """Process JSON data and create PPT slides"""
        # Add title slide
        self.add_title_slide(query.upper())

        for section_key, section_data in json_data.items():
            section_title = section_data['title']
            current_items = []

            for item in section_data['items']:
                if 'content' in item:
                    # Handle regular content
                    content_texts = []
                    for content_item in item['content']:
                        if isinstance(content_item, dict):
                            if content_item['type'] == 'code':
                                # First add any accumulated content
                                if content_texts:
                                    self.add_content_slide(f"{section_title} - {item['title']}", content_texts)
                                    content_texts = []
                                # Then add code slide
                                self.add_code_slide(f"{section_title} - {item['title']}", content_item['content'])
                            elif content_item['type'] == 'text':
                                content_texts.append(content_item['content'])

                    if content_texts:
                        self.add_content_slide(f"{section_title} - {item['title']}", content_texts)

                # Handle subitems
                if 'subitems' in item:
                    subitem_texts = []
                    for subitem in item['subitems']:
                        if 'content' in subitem:
                            for content in subitem['content']:
                                if isinstance(content, dict) and content['type'] == 'text':
                                    title = f"{subitem.get('title', '')}"
                                    if title:
                                        subitem_texts.append(f"{title}: {content['content']}")
                                    else:
                                        subitem_texts.append(content['content'])

                    if subitem_texts:
                        self.add_content_slide(f"{section_title} - {item['title']}", subitem_texts, is_subsection=True)

    def save(self, filename):
        """Save the presentation"""
        self.prs.save(filename)

def create_presentation(json_file_path, output_pptx_path):
    """Main function to create presentation from JSON"""
    # Read JSON data
    with open(json_file_path, 'r') as f:
        json_data = json.load(f)

    # Create and save presentation
    ppt_gen = PPTGenerator()
    ppt_gen.process_json_to_ppt(json_data)
    ppt_gen.save(output_pptx_path)
    print(f"Presentation saved as {output_pptx_path}")

# Usage example
if __name__ == "__main__":
    create_presentation("output.json", query.upper()+".pptx")

Presentation saved as LSTM.pptx


##Image PPT

In [86]:
import json
from io import BytesIO
import os
import requests
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_AUTO_SIZE
from PIL import Image

class PPTGenerator:
    def __init__(self, template_path="/content/Crop.pptx"):
        # Load template if available, else create a new presentation.
        if os.path.exists(template_path):
            self.prs = Presentation(template_path)
        else:
            print(f"Template {template_path} not found. Creating a new presentation.")
            self.prs = Presentation()
        self.slide_width = self.prs.slide_width  # in EMUs
        self.slide_height = self.prs.slide_height  # in EMUs
        self.slide_count = 0

        # Color scheme
        self.colors = {
            'title': RGBColor(44, 62, 80),      # Dark blue
            'subtitle': RGBColor(52, 73, 94),    # Lighter blue
            'body': RGBColor(44, 62, 80),        # Dark blue
            'code': RGBColor(46, 204, 113),      # Green
            'slide_number': RGBColor(149, 165, 166)  # Gray
        }

    def add_slide_number(self, slide):
        """Add slide number to the bottom right corner."""
        self.slide_count += 1
        txBox = slide.shapes.add_textbox(Inches(9), Inches(6.5), Inches(1), Inches(0.5))
        tf = txBox.text_frame
        p = tf.paragraphs[0]
        p.text = f"Slide {self.slide_count}"
        p.alignment = PP_ALIGN.RIGHT
        p.font.size = Pt(12)
        p.font.color.rgb = self.colors['slide_number']

    def add_title_slide(self, title):
        """Add title slide."""
        slide_layout = self.prs.slide_layouts[0]
        slide = self.prs.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)

        self.add_slide_number(slide)
        return slide

    def format_bullet_point(self, paragraph, level=0):
        """Format bullet point with proper indentation and style."""
        paragraph.level = level
        paragraph.font.size = Pt(25 - level)  # Decrease font size for deeper levels
        paragraph.font.color.rgb = self.colors['body']
        return paragraph

    def add_content_slide(self, title, content_items, is_subsection=False):
        """Add content slide with bullet points and overflow management."""
        slide_layout = self.prs.slide_layouts[1]
        current_slide = self.prs.slides.add_slide(slide_layout)

        # Add title
        title_shape = current_slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        # Add content from placeholder
        content_shape = current_slide.placeholders[1]
        tf = content_shape.text_frame
        tf.word_wrap = True
        tf.clear()

        current_height = 0
        max_height = Inches(5)  # Maximum content height

        for item in content_items:
            # Rough estimation of text height
            text_height = len(item.split()) * Pt(25) * 0.3

            if current_height + text_height > max_height:
                # Create new slide if content overflows
                current_slide = self.prs.slides.add_slide(slide_layout)
                title_shape = current_slide.shapes.title
                title_shape.text = f"{title} (continued)"
                title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                title_shape.text_frame.paragraphs[0].font.size = Pt(32)

                content_shape = current_slide.placeholders[1]
                tf = content_shape.text_frame
                tf.word_wrap = True
                current_height = 0

                self.add_slide_number(current_slide)

            p = tf.add_paragraph()
            p.text = item
            self.format_bullet_point(p, 1 if is_subsection else 0)
            current_height += text_height

        self.add_slide_number(current_slide)
        return current_slide

    def add_code_slide(self, title, code):
        """Add code slide with proper formatting."""
        slide_layout = self.prs.slide_layouts[1]
        slide = self.prs.slides.add_slide(slide_layout)

        # Add title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        # Add code textbox
        left = Inches(2)
        top = Inches(2.6)
        width = Inches(8)
        height = Inches(5)

        textbox = slide.shapes.add_textbox(left, top, width, height)
        tf = textbox.text_frame
        tf.word_wrap = True

        # Split code into chunks if too long
        code_lines = code.split('\n')
        lines_per_slide = 20

        for i in range(0, len(code_lines), lines_per_slide):
            if i > 0:
                slide = self.prs.slides.add_slide(slide_layout)
                title_shape = slide.shapes.title
                title_shape.text = f"{title} (continued)"
                title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                textbox = slide.shapes.add_textbox(left, top, width, height)
                tf = textbox.text_frame
                tf.word_wrap = True

            chunk = '\n'.join(code_lines[i:i + lines_per_slide])
            p = tf.add_paragraph()
            p.text = chunk
            p.font.name = 'Courier New'
            p.font.size = Pt(11)
            p.font.color.rgb = self.colors['code']

            self.add_slide_number(slide)

        return slide

    def add_image_slide(self, title, image_url):
        """
        Add an image slide with a title and a centered image.
        Downloads the image from the provided URL.
        """
        # Use a blank layout if available; if not, fall back to layout 0.
        if len(self.prs.slide_layouts) > 6:
            blank_layout = self.prs.slide_layouts[6]
        else:
            blank_layout = self.prs.slide_layouts[0]
        slide = self.prs.slides.add_slide(blank_layout)

        # Add title textbox at the top of the slide.
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), self.slide_width - Inches(1), Inches(1))
        title_tf = title_box.text_frame
        title_p = title_tf.add_paragraph()
        title_p.text = title
        title_p.font.size = Pt(32)
        title_p.font.color.rgb = self.colors['title']

        # Debug: Print the image URL being processed.
        print(f"Processing image URL: {image_url}")

        # Download the image.
        response = requests.get(image_url)
        if response.status_code != 200:
            print(f"Failed to download image. Status code: {response.status_code} for URL: {image_url}")
            return slide

        image_data = BytesIO(response.content)

        # Convert WEBP images to PNG if needed.
        if image_url.lower().endswith('.webp'):
            image_data = convert_webp_to_png(image_data)
            if not image_data:
                print(f"Skipping invalid image: {image_url}")
                return slide

        # Calculate image placement:
        image_left = Inches(1)
        image_top = Inches(1.5)
        image_width = self.slide_width - Inches(2)  # full width minus margins
        image_height = self.slide_height - Inches(2.5)  # leave margin at bottom

        # Debug: Print placement details.
        print(f"Placing image at left: {image_left}, top: {image_top}, width: {image_width}, height: {image_height}")

        try:
            slide.shapes.add_picture(image_data, image_left, image_top, width=image_width, height=image_height)
            print("Image added successfully!")
        except Exception as e:
            print(f"Error adding image: {e}")

        self.add_slide_number(slide)
        return slide

    def add_images_from_set(self, image_urls_set, title_prefix="Extra Image"):
        """
        Iterate over the set of image URLs and add each as a separate image slide.
        """
        for idx, image_url in enumerate(image_urls_set, start=1):
            slide_title = f"{title_prefix} {idx}"
            self.add_image_slide(slide_title, image_url)

    def process_json_to_ppt(self, json_data, query, image_urls=None):
        """Process JSON data and create PPT slides.
           If image_urls (a list or set) is provided, insert an image slide after processing each content item.
        """
        # If image_urls is a set, convert it to a list so we can pop from it.
        if image_urls and isinstance(image_urls, set):
            image_urls = list(image_urls)

        # Add title slide.
        self.add_title_slide(query.upper())

        for section_key, section_data in json_data.items():
            section_title = section_data['title']
            for item in section_data['items']:
                if 'content' in item:
                    content_texts = []
                    for content_item in item['content']:
                        if isinstance(content_item, dict):
                            if content_item['type'] == 'code':
                                # If there is any pending text content, add it first.
                                if content_texts:
                                    self.add_content_slide(f"{section_title} - {item['title']}", content_texts)
                                    content_texts = []
                                self.add_code_slide(f"{section_title} - {item['title']}", content_item['content'])
                            elif content_item['type'] == 'text':
                                content_texts.append(content_item['content'])
                    if content_texts:
                        self.add_content_slide(f"{section_title} - {item['title']}", content_texts)

                    # After processing the item, if an image URL is available, add an image slide.
                    if image_urls and len(image_urls) > 0:
                        next_image_url = image_urls.pop(0)
                        self.add_image_slide(f"{section_title} - {item['title']} Image", next_image_url)

                if 'subitems' in item:
                    subitem_texts = []
                    for subitem in item['subitems']:
                        if 'content' in subitem:
                            for content in subitem['content']:
                                if isinstance(content, dict) and content['type'] == 'text':
                                    title = f"{subitem.get('title', '')}"
                                    if title:
                                        subitem_texts.append(f"{title}: {content['content']}")
                                    else:
                                        subitem_texts.append(content['content'])
                    if subitem_texts:
                        self.add_content_slide(f"{section_title} - {item['title']}", subitem_texts, is_subsection=True)
                        # Optionally insert an image slide after subitems, too.
                        if image_urls and len(image_urls) > 0:
                            next_image_url = image_urls.pop(0)
                            self.add_image_slide(f"{section_title} - {item['title']} Subitem Image", next_image_url)

    def save(self, filename):
        """Save the presentation."""
        self.prs.save(filename)

def convert_webp_to_png(image_data):
    """Convert WEBP image data to PNG."""
    try:
        with Image.open(image_data) as img:
            png_buffer = BytesIO()
            img.save(png_buffer, format="PNG")
            png_buffer.seek(0)
            return png_buffer
    except Exception as e:
        print(f"Error converting WEBP to PNG: {e}")
        return None

def create_presentation(json_file_path, output_pptx_path, query, image_urls=None):
    """
    Main function to create a presentation from JSON and intersperse extra image slides
    from a set (or list) of image URLs.
    """
    with open(json_file_path, 'r') as f:
        json_data = json.load(f)

    ppt_gen = PPTGenerator()
    # Pass the image_urls container so they are inserted between content slides.
    ppt_gen.process_json_to_ppt(json_data, query, image_urls=image_urls)

    ppt_gen.save(output_pptx_path)
    print(f"Presentation saved as {output_pptx_path}")

if __name__ == "__main__":

    output_filename = query.upper() + ".pptx"



    create_presentation("output_merged.json", output_filename, query, images)


Processing image URL: https://media.geeksforgeeks.org/wp-content/uploads/20230529103946/Bidirectional-LSTM-(1)-660.jpg
Placing image at left: 914400, top: 1371600, width: 10363200, height: 4572000
Image added successfully!
Processing image URL: https://media.geeksforgeeks.org/wp-content/uploads/20240425175947/Capture.PNG
Placing image at left: 914400, top: 1371600, width: 10363200, height: 4572000
Image added successfully!
Processing image URL: https://media.geeksforgeeks.org/wp-content/uploads/20240208053129/lstm.webp
Placing image at left: 914400, top: 1371600, width: 10363200, height: 4572000
Image added successfully!
Processing image URL: https://media.geeksforgeeks.org/wp-content/uploads/20240208104902/bruh.webp
Placing image at left: 914400, top: 1371600, width: 10363200, height: 4572000
Image added successfully!
Presentation saved as LSTM.pptx


In [ ]:
import json
import os
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_AUTO_SIZE

class PPTGenerator:
    def __init__(self, template_path):
        print(f"Initializing PPT Generator with template: {template_path}")
        self.prs = Presentation(template_path)
        self.slide_width = self.prs.slide_width
        self.slide_height = self.prs.slide_height
        self.slide_count = 0

        # Color scheme
        self.colors = {
            'title': RGBColor(44, 62, 80),      # Dark blue
            'subtitle': RGBColor(52, 73, 94),    # Lighter blue
            'body': RGBColor(44, 62, 80),        # Dark blue
            'code': RGBColor(46, 204, 113),      # Green
            'slide_number': RGBColor(149, 165, 166)  # Gray
        }

    def add_slide_number(self, slide):
        """Add slide number to the bottom right corner"""
        self.slide_count += 1
        txBox = slide.shapes.add_textbox(
            Inches(9), Inches(6.5), Inches(1), Inches(0.5))
        tf = txBox.text_frame
        p = tf.paragraphs[0]
        p.text = f"Slide {self.slide_count}"
        p.alignment = PP_ALIGN.RIGHT
        p.font.size = Pt(12)
        p.font.color.rgb = self.colors['slide_number']

    def add_title_slide(self, title):
        """Add title slide"""
        print(f"Adding title slide: {title}")
        slide_layout = self.prs.slide_layouts[0]
        slide = self.prs.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)

        self.add_slide_number(slide)
        return slide

    def format_bullet_point(self, paragraph, level=0):
        """Format bullet point with proper indentation and style"""
        paragraph.level = level
        paragraph.font.size = Pt(25 - level)  # Decrease font size for deeper levels
        paragraph.font.color.rgb = self.colors['body']
        return paragraph

    def add_content_slide(self, title, content_items, is_subsection=False):
        """Add content slide with proper bullet points and overflow management"""
        print(f"Adding content slide: {title}")
        slide_layout = self.prs.slide_layouts[1]
        current_slide = self.prs.slides.add_slide(slide_layout)

        # Add title
        title_shape = current_slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        # Add content
        content_shape = current_slide.placeholders[1]
        tf = content_shape.text_frame
        tf.word_wrap = True
        tf.clear()

        current_height = 0
        max_height = Inches(5)

        for item in content_items:
            text_height = len(str(item).split()) * Pt(25) * 0.3

            if current_height + text_height > max_height:
                current_slide = self.prs.slides.add_slide(slide_layout)
                title_shape = current_slide.shapes.title
                title_shape.text = f"{title} (continued)"
                title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                title_shape.text_frame.paragraphs[0].font.size = Pt(32)

                content_shape = current_slide.placeholders[1]
                tf = content_shape.text_frame
                tf.word_wrap = True
                current_height = 0
                self.add_slide_number(current_slide)

            p = tf.add_paragraph()
            p.text = str(item)
            self.format_bullet_point(p, 1 if is_subsection else 0)
            current_height += text_height

        self.add_slide_number(current_slide)
        return current_slide

    def add_code_slide(self, title, code):
        """Add code slide with proper formatting"""
        print(f"Adding code slide: {title}")
        slide_layout = self.prs.slide_layouts[1]
        slide = self.prs.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        left = Inches(2)
        top = Inches(2.6)
        width = Inches(8)
        height = Inches(5)

        textbox = slide.shapes.add_textbox(left, top, width, height)
        tf = textbox.text_frame
        tf.word_wrap = True

        code_lines = str(code).split('\n')
        lines_per_slide = 20

        for i in range(0, len(code_lines), lines_per_slide):
            if i > 0:
                slide = self.prs.slides.add_slide(slide_layout)
                title_shape = slide.shapes.title
                title_shape.text = f"{title} (continued)"
                title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                textbox = slide.shapes.add_textbox(left, top, width, height)
                tf = textbox.text_frame
                tf.word_wrap = True

            chunk = '\n'.join(code_lines[i:i + lines_per_slide])
            p = tf.add_paragraph()
            p.text = chunk
            p.font.name = 'Courier New'
            p.font.size = Pt(11)
            p.font.color.rgb = self.colors['code']

            self.add_slide_number(slide)

        return slide

    def add_equation_slide(self, title, equations):
        """Add slide with equation images"""
        print(f"\nAdding equation slide: {title}")
        print(f"Number of equations to add: {len(equations)}")

        slide_layout = self.prs.slide_layouts[1]
        slide = self.prs.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
        title_shape.text_frame.paragraphs[0].font.size = Pt(32)

        left = Inches(2)
        top = Inches(2)
        max_height = Inches(5)
        current_top = top

        for equation_path in equations:
            print(f"Attempting to add equation from: {equation_path}")
            try:
                if not os.path.exists(equation_path):
                    print(f"Error: File not found - {equation_path}")
                    continue

                img = slide.shapes.add_picture(
                    equation_path,
                    left,
                    current_top,
                    width=Inches(6)
                )
                print(f"Successfully added equation image: {equation_path}")
                current_top += img.height + Inches(0.5)

                if current_top > top + max_height:
                    print("Creating new slide for overflow equations")
                    slide = self.prs.slides.add_slide(slide_layout)
                    title_shape = slide.shapes.title
                    title_shape.text = f"{title} (continued)"
                    title_shape.text_frame.paragraphs[0].font.color.rgb = self.colors['title']
                    title_shape.text_frame.paragraphs[0].font.size = Pt(32)
                    current_top = top
                    self.add_slide_number(slide)

            except Exception as e:
                print(f"Error adding equation image {equation_path}: {str(e)}")
                print(f"Exception type: {type(e)}")
                import traceback
                traceback.print_exc()

        self.add_slide_number(slide)
        return slide

    def process_json_to_ppt(self, json_data, title):
        """Process JSON data and create PPT slides"""
        print("\nStarting PPT Generation...")
        print(f"Processing title: {title}")

        self.add_title_slide(title)

        for section_key, section_data in json_data.items():
            print(f"\nProcessing section: {section_key}")

            if section_key == "metadata":
                print("Skipping metadata section")
                continue

            section_title = section_data['title']
            print(f"Section title: {section_title}")

            for item in section_data['items']:
                print(f"\nProcessing item: {item.get('title', 'Untitled')}")

                if 'content' in item:
                    content_texts = []
                    equation_images = []

                    for content_item in item['content']:
                        if isinstance(content_item, dict):
                            content_type = content_item.get('type', 'unknown')
                            print(f"Found content of type: {content_type}")

                            if content_type == 'code':
                                if content_texts:
                                    self.add_content_slide(f"{section_title} - {item.get('title', '')}", content_texts)
                                    content_texts = []
                                self.add_code_slide(f"{section_title} - {item.get('title', '')}", content_item['content'])

                            elif content_type == 'equation_image':
                                print(f"Found equation image: {content_item['content']}")
                                equation_images.append(content_item['content'])

                            elif content_type == 'text':
                                content_texts.append(content_item['content'])

                    if content_texts:
                        print(f"Adding content slide with {len(content_texts)} text items")
                        self.add_content_slide(f"{section_title} - {item.get('title', '')}", content_texts)

                    if equation_images:
                        print(f"Adding equation slide with {len(equation_images)} equations")
                        self.add_equation_slide(f"{section_title} - {item.get('title', '')}", equation_images)
                        equation_images = []

                if 'subitems' in item:
                    print(f"Processing subitems for: {item.get('title', 'Untitled')}")
                    subitem_texts = []
                    subitem_equations = []

                    for subitem in item['subitems']:
                        print(f"Processing subitem: {subitem.get('title', 'Untitled')}")
                        if 'content' in subitem:
                            for content in subitem['content']:
                                if isinstance(content, dict):
                                    content_type = content.get('type', 'unknown')
                                    print(f"Found subitem content of type: {content_type}")

                                    if content_type == 'equation_image':
                                        print(f"Found subitem equation: {content['content']}")
                                        subitem_equations.append(content['content'])
                                    elif content_type == 'text':
                                        title = subitem.get('title', '')
                                        if title:
                                            subitem_texts.append(f"{title}: {content['content']}")
                                        else:
                                            subitem_texts.append(content['content'])

                    if subitem_texts:
                        print(f"Adding subitem content slide with {len(subitem_texts)} items")
                        self.add_content_slide(f"{section_title} - {item.get('title', '')}",
                                            subitem_texts,
                                            is_subsection=True)

                    if subitem_equations:
                        print(f"Adding subitem equation slide with {len(subitem_equations)} equations")
                        self.add_equation_slide(f"{section_title} - {item.get('title', '')}",
                                             subitem_equations)

        print("\nPPT Generation completed!")

    def save(self, filename):
        """Save the presentation"""
        print(f"\nSaving presentation to: {filename}")
        self.prs.save(filename)
        print("Presentation saved successfully!")

def create_presentation(json_file_path, output_pptx_path, title, template_path):
    """Main function to create presentation from JSON"""
    print(f"\nStarting presentation creation...")
    print(f"Input JSON: {json_file_path}")
    print(f"Output PPTX: {output_pptx_path}")
    print(f"Template: {template_path}")

    try:
        with open(json_file_path, 'r') as f:
            json_data = json.load(f)
            print("JSON file loaded successfully")
            print(f"Number of sections: {len(json_data)}")
    except Exception as e:
        print(f"Error loading JSON file: {str(e)}")
        return

    try:
        ppt_gen = PPTGenerator(template_path)
        print("PPT Generator initialized")

        ppt_gen.process_json_to_ppt(json_data, title)
        print("Content processing completed")

        ppt_gen.save(output_pptx_path)
        print(f"Presentation saved as {output_pptx_path}")
    except Exception as e:
        print(f"Error creating presentation: {str(e)}")
        import traceback
        traceback.print_exc()

# Usage example
if __name__ == "__main__":
    template_path = "/content/Crop.pptx"  # Update this path to your template
    json_file_path = "output_updated.json"
    output_pptx_path = query.upper()+".pptx"
    presentation_title = query.upper()

    create_presentation(
        json_file_path=json_file_path,
        output_pptx_path=output_pptx_path,
        title=presentation_title,
        template_path=template_path
    )


Starting presentation creation...
Input JSON: output_updated.json
Output PPTX: MUTEX LOCKS.pptx
Template: /content/Crop.pptx
JSON file loaded successfully
Number of sections: 7
Initializing PPT Generator with template: /content/Crop.pptx
PPT Generator initialized

Starting PPT Generation...
Processing title: MUTEX LOCKS
Adding title slide: MUTEX LOCKS

Processing section: Section 1
Section title: Concept Overview

Processing item: Core definition and purpose
Found content of type: text
Adding content slide with 1 text items
Adding content slide: Concept Overview - Core definition and purpose

Processing item: Key principles
Found content of type: text
Adding content slide with 1 text items
Adding content slide: Concept Overview - Key principles

Processing item: Relationship to broader computing concepts
Found content of type: text
Adding content slide with 1 text items
Adding content slide: Concept Overview - Relationship to broader computing concepts

Processing section: Section 2
S

#Convert text file to .tex and then .tex to pdf

In [ ]:
import re
import os
import subprocess

class LatexConverter:
    def __init__(self):
        self.template = r"""\documentclass{article}
\usepackage{amsmath}
\usepackage{listings}
\usepackage{xcolor}
\usepackage{enumitem}
\usepackage{titlesec}
\usepackage{geometry}

\geometry{
    a4paper,
    margin=1in
}

% Section formatting
\titleformat{\section}
    {\Large\bfseries}
    {\thesection}
    {1em}
    {}

\titleformat{\subsection}
    {\large\bfseries}
    {\thesubsection}
    {1em}
    {}

% List settings
\setlist[enumerate]{label=\arabic*.}

% Code listing settings
\lstset{
    backgroundcolor=\color{gray!10},
    basicstyle=\ttfamily\footnotesize,
    breaklines=true,
    frame=single,
    numbers=left,
    numberstyle=\tiny,
    keywordstyle=\color{blue},
    commentstyle=\color{green!60!black},
    stringstyle=\color{red},
    showstringspaces=false,
    tabsize=4
}

\begin{document}
\pagestyle{plain}
"""

    def _clean_text(self, text: str) -> str:
        """Clean text while preserving math content."""
        if '$' in text:  # Contains math expressions
            return text

        text = text.replace('\\', r'\textbackslash{}')
        text = text.replace('_', r'\_')
        text = text.replace('^', r'\^{}')
        text = text.replace('~', r'\textasciitilde{}')
        text = text.replace('#', r'\#')
        text = text.replace('%', r'\%')
        text = text.replace('&', r'\&')
        text = text.replace('<', r'\textless{}')
        text = text.replace('>', r'\textgreater{}')

        # Handle bold text
        text = re.sub(r'\*\*(.*?)\*\*', r'\\textbf{\1}', text)

        return text

    def convert_to_tex(self, content: str) -> str:
        """Convert markdown content to LaTeX."""
        output = [self.template]
        lines = content.split('\n')
        in_code_block = False
        i = 0

        while i < len(lines):
            line = lines[i].strip()

            if not line:
                i += 1
                continue

            # Handle section headers
            if line.startswith('## '):
                title = re.search(r'\*\*(.*?)\*\*', line)
                if title:
                    output.append(f"\\section{{{title.group(1)}}}")
                i += 1
                continue

            # Handle code blocks
            if line.startswith('```'):
                if 'python' in line.lower():
                    output.append('\\begin{lstlisting}[language=Python]')
                else:
                    output.append('\\begin{lstlisting}')
                i += 1
                while i < len(lines) and not lines[i].strip().startswith('```'):
                    output.append(lines[i])
                    i += 1
                output.append('\\end{lstlisting}')
                i += 1
                continue

            # Handle numbered lists
            if re.match(r'^\d+\.\s', line):
                output.append('\\begin{enumerate}[resume*]')
                while i < len(lines) and re.match(r'^\d+\.\s', lines[i].strip()):
                    item = re.sub(r'^\d+\.\s', '', lines[i].strip())
                    item = self._clean_text(item)
                    output.append(f"\\item {item}")
                    i += 1
                output.append('\\end{enumerate}')
                continue

            # Regular line
            text = self._clean_text(line)
            if text:
                output.append(text + "\\\\")
            i += 1

        output.append("\\end{document}")
        return '\n'.join(output)

    def create_pdf(self, content: str, output_filename: str = "output") -> str:
        """Convert content to LaTeX and compile to PDF."""
        try:
            # Convert content to LaTeX
            tex_content = self.convert_to_tex(content)

            # Write TEX file
            tex_file = f"{output_filename}.tex"
            with open(tex_file, "w", encoding='utf-8') as f:
                f.write(tex_content)

            # Compile twice for references
            for _ in range(2):
                result = subprocess.run(
                    ['pdflatex', '-interaction=nonstopmode', tex_file],
                    capture_output=True,
                    text=True,
                    check=False
                )
                if result.returncode != 0:
                    with open(f"{output_filename}.log", "r") as f:
                        log = f.read()
                    raise RuntimeError(f"PDF compilation failed. Check {output_filename}.log for details")

            # Clean up auxiliary files
            if os.path.exists(f"{output_filename}.pdf"):
                for ext in ['.aux', '.out']:
                    if os.path.exists(f"{output_filename}{ext}"):
                        os.remove(f"{output_filename}{ext}")
                return f"{output_filename}.pdf"
            else:
                raise RuntimeError("PDF compilation failed")

        except Exception as e:
            raise RuntimeError(f"An error occurred: {str(e)}")

# Example usage
if __name__ == "__main__":
    try:
        converter = LatexConverter()

        # Read input file
        with open("content.txt", "r", encoding='utf-8') as f:
            content = f.read()

        # Convert to PDF
        pdf_file = converter.create_pdf(content, query.upper())
        print(f"Successfully created {pdf_file}")

    except Exception as e:
        print(f"Error: {str(e)}")

Error: An error occurred: PDF compilation failed. Check SVM.log for details
